In [1]:
# ! pip install pandas biopython

In [2]:
# Import required libraries
import os
import glob
import pandas as pd
from Bio import SeqIO

# --- Configuration ---
# Directory where your files are located (adjust if needed)
# If files are in the current working directory, you can leave this as is
data_dir = "."

# File patterns
alignment_file = os.path.join(data_dir, "alignment.fasta")
tab_file_pattern = os.path.join(data_dir, "plantCARE_output_*.tab")

# Column names for the plantCARE output files
column_names = [
    "Experiment", "Motif", "Motif_Sequence", 
    "Start", "Length", "Strand", "Organism", "Function"
]

# --- Step 1: Parse the alignment and build position mappings ---
print("Parsing alignment file and building position mappings...")

# Dictionary to store mapping: seq_id -> list where index = ungapped position (0-based),
# value = alignment column index (0-based)
# We'll convert to 1-based later when writing output
ungapped_to_alignment = {}

# Read alignment
alignment_records = SeqIO.to_dict(SeqIO.parse(alignment_file, "fasta"))

for seq_id, record in alignment_records.items():
    seq_str = str(record.seq)
    mapping = []
    alignment_pos = 0  # 0-based alignment column index
    
    for char in seq_str:
        if char != "-":
            # This is a real base; record the current alignment column
            mapping.append(alignment_pos)
        # Always increment alignment position (including for gaps)
        alignment_pos += 1
    
    ungapped_to_alignment[seq_id] = mapping
    print(f"Processed sequence '{seq_id}': {len(mapping)} ungapped positions, alignment length = {len(seq_str)}")

# --- Step 2: Process each plantCARE output file ---
print("\nProcessing plantCARE output files...")

for tab_file in glob.glob(tab_file_pattern):
    # Extract sequence name from filename
    # Expected format: plantCARE_output_<seq_name>.tab
    basename = os.path.basename(tab_file)
    if basename.startswith("plantCARE_output_") and basename.endswith(".tab"):
        seq_name = basename[len("plantCARE_output_"):-len(".tab")]
    else:
        print(f"Skipping unexpected filename: {basename}")
        continue
    
    # Check if we have this sequence in the alignment
    if seq_name not in ungapped_to_alignment:
        print(f"Warning: Sequence '{seq_name}' not found in alignment. Skipping {basename}")
        continue
    
    # Read the tab file
    try:
        df = pd.read_csv(tab_file, sep="\t", header=None, names=column_names)
    except Exception as e:
        print(f"Error reading {tab_file}: {e}")
        continue
    
    # Get the mapping for this sequence
    mapping = ungapped_to_alignment[seq_name]
    max_ungapped_pos = len(mapping)
    
    # Convert Start positions (1-based in input) to alignment columns (1-based in output)
    def convert_start(start):
        # Input 'Start' is 1-based position in ungapped sequence
        ungapped_index = start - 1  # convert to 0-based
        if ungapped_index < 0 or ungapped_index >= max_ungapped_pos:
            print(f"Warning: Start position {start} out of range for sequence '{seq_name}' (length={max_ungapped_pos})")
            return None
        alignment_col_0based = mapping[ungapped_index]
        return alignment_col_0based + 1  # convert back to 1-based for output
    
    # Apply conversion
    df['Start_Aligned'] = df['Start'].apply(convert_start)
    
    # Drop rows where conversion failed (out of range)
    original_count = len(df)
    df = df.dropna(subset=['Start_Aligned'])
    df['Start_Aligned'] = df['Start_Aligned'].astype(int)
    
    if len(df) < original_count:
        print(f"Warning: Dropped {original_count - len(df)} rows from {basename} due to out-of-range positions")
    
    # Replace the original 'Start' column with aligned positions
    df = df.drop(columns=['Start']).rename(columns={'Start_Aligned': 'Start'})
    
    # Reorder columns to match original order (with updated Start)
    df = df[column_names]
    
    # Save to new file
    output_file = os.path.join(data_dir, f"plantCARE_output_aligned_{seq_name}.tab")
    df.to_csv(output_file, sep="\t", index=False, header=False)
    print(f"Saved aligned positions for '{seq_name}' to {output_file} ({len(df)} rows)")

print("\nDone!")

Parsing alignment file and building position mappings...
Processed sequence 'Dublet': 933 ungapped positions, alignment length = 945
Processed sequence 'CS': 945 ungapped positions, alignment length = 945

Processing plantCARE output files...
Saved aligned positions for 'CS' to .\plantCARE_output_aligned_CS.tab (74 rows)
Saved aligned positions for 'Dublet' to .\plantCARE_output_aligned_Dublet.tab (77 rows)

Done!


In [4]:
# Import required libraries
import os
import glob
import pandas as pd
from Bio import SeqIO

# --- Configuration ---
data_dir = "."
alignment_file = os.path.join(data_dir, "alignment.fasta")
aligned_tab_pattern = os.path.join(data_dir, "plantCARE_output_aligned_*.tab")

# Column names (excluding 'Experiment' for comparison)
compare_columns = [
    "Motif", "Motif_Sequence", "Start", "Length", "Strand", "Organism", "Function"
]

# --- Step 1: Get sequence names from alignment ---
print("Reading sequence names from alignment...")

with open(alignment_file, "r") as f:
    seq_names = []
    for line in f:
        if line.startswith(">"):
            seq_name = line[1:].strip().split(" ")[0]
            seq_names.append(seq_name)

if len(seq_names) != 2:
    raise ValueError(f"Expected exactly 2 sequences in alignment, but found {len(seq_names)}: {seq_names}")

seq1, seq2 = seq_names
print(f"Comparing sequences: '{seq1}' vs '{seq2}'")

# --- Step 2: Load aligned tables ---
def load_aligned_table(seq_name):
    filename = os.path.join(data_dir, f"plantCARE_output_aligned_{seq_name}.tab")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Aligned table not found: {filename}")
    
    # Read with all columns (including Experiment)
    df = pd.read_csv(
        filename, 
        sep="\t", 
        header=None, 
        names=["Experiment"] + compare_columns
    )
    return df

print("Loading aligned tables...")
df1 = load_aligned_table(seq1)
df2 = load_aligned_table(seq2)

print(f"Loaded {len(df1)} entries for '{seq1}'")
print(f"Loaded {len(df2)} entries for '{seq2}'")

# --- Step 3: Perform anti-joins ---
print("\nPerforming anti-joins...")

# Function to perform anti-join
def anti_join(left_df, right_df, on_columns):
    """Return rows in left_df that are NOT present in right_df based on on_columns"""
    # Reset index to preserve original row order
    left_df = left_df.reset_index(drop=True)
    right_df = right_df.reset_index(drop=True)
    
    # Merge with indicator
    merged = left_df.merge(
        right_df[on_columns], 
        on=on_columns, 
        how='left', 
        indicator=True
    )
    
    # Keep only rows that were NOT in right_df
    result = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
    
    # Restore original index type
    result = result.reset_index(drop=True)
    return result

# Find elements in seq1 but NOT in seq2
unique_to_seq1 = anti_join(df1, df2, compare_columns)
print(f"Found {len(unique_to_seq1)} elements unique to '{seq1}'")

# Find elements in seq2 but NOT in seq1
unique_to_seq2 = anti_join(df2, df1, compare_columns)
print(f"Found {len(unique_to_seq2)} elements unique to '{seq2}'")

# --- Step 4: Save filtered results ---
def save_filtered_table(df, seq_name):
    output_file = os.path.join(data_dir, f"plantCARE_output_aligned_filtered_{seq_name}.tab")
    df.to_csv(output_file, sep="\t", index=False, header=False)
    print(f"Saved filtered results for '{seq_name}' to {output_file}")

print("\nSaving filtered files...")
save_filtered_table(unique_to_seq1, seq1)
save_filtered_table(unique_to_seq2, seq2)

# --- Optional: Display summary ---
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Original entries - {seq1}: {len(df1)}")
print(f"Original entries - {seq2}: {len(df2)}")
print(f"Unique to {seq1}: {len(unique_to_seq1)}")
print(f"Unique to {seq2}: {len(unique_to_seq2)}")
print(f"Common elements: {len(df1) - len(unique_to_seq1)}")

# Optional: Show examples of unique elements
if len(unique_to_seq1) > 0:
    print(f"\nFirst 3 unique elements in {seq1}:")
    display_cols = ["Motif", "Motif_Sequence", "Start", "Strand"]
    print(unique_to_seq1[display_cols].head(3).to_string(index=False))

if len(unique_to_seq2) > 0:
    print(f"\nFirst 3 unique elements in {seq2}:")
    print(unique_to_seq2[display_cols].head(3).to_string(index=False))

Reading sequence names from alignment...
Comparing sequences: 'Dublet' vs 'CS'
Loading aligned tables...
Loaded 77 entries for 'Dublet'
Loaded 74 entries for 'CS'

Performing anti-joins...
Found 4 elements unique to 'Dublet'
Found 1 elements unique to 'CS'

Saving filtered files...
Saved filtered results for 'Dublet' to .\plantCARE_output_aligned_filtered_Dublet.tab
Saved filtered results for 'CS' to .\plantCARE_output_aligned_filtered_CS.tab

SUMMARY
Original entries - Dublet: 77
Original entries - CS: 74
Unique to Dublet: 4
Unique to CS: 1
Common elements: 73

First 3 unique elements in Dublet:
      Motif Motif_Sequence  Start Strand
TGACG-motif          TGACG    129      +
CGTCA-motif          CGTCA    129      -
       as-1          TGACG    129      +

First 3 unique elements in CS:
     Motif Motif_Sequence  Start Strand
Unnamed__4           CTCC     65      +


In [51]:
import os
import pandas as pd
from Bio import SeqIO
from IPython.display import HTML, display
import random

def visualize_differential_elements_html(
    start_col=50, 
    end_col=150,
    data_dir=".",
    alignment_file="alignment.fasta",
    filtered_suffix="_filtered",
    output_file=None
):
    """
    Final version with perfect alignment and clean display.
    """
    
    # --- Step 1: Get sequence names from alignment ---
    alignment_path = os.path.join(data_dir, alignment_file)
    with open(alignment_path, "r") as f:
        seq_names = []
        for line in f:
            if line.startswith(">"):
                seq_names.append(line[1:].strip().split(" ")[0])
    
    if len(seq_names) != 2:
        raise ValueError(f"Expected exactly 2 sequences, found {len(seq_names)}")
    
    seq1_name, seq2_name = seq_names
    
    # --- Step 2: Load alignment sequences ---
    alignment_dict = SeqIO.to_dict(SeqIO.parse(alignment_path, "fasta"))
    seq1_full = str(alignment_dict[seq1_name].seq)
    seq2_full = str(alignment_dict[seq2_name].seq)
    
    # Validate column range
    alignment_length = len(seq1_full)
    if len(seq2_full) != alignment_length:
        raise ValueError("Sequences in alignment have different lengths!")
    
    if start_col < 1 or end_col > alignment_length or start_col > end_col:
        raise ValueError(f"Invalid column range: {start_col}-{end_col} (alignment length: {alignment_length})")
    
    # Extract the region (convert to 0-based indexing)
    start_idx = start_col - 1
    end_idx = end_col
    seq1_region = seq1_full[start_idx:end_idx]
    seq2_region = seq2_full[start_idx:end_idx]
    region_length = len(seq1_region)
    
    # --- Step 3: Load filtered element data ---
    def load_filtered_elements(seq_name):
        filename = os.path.join(
            data_dir, 
            f"plantCARE_output_aligned{filtered_suffix}_{seq_name}.tab"
        )
        if not os.path.exists(filename):
            return pd.DataFrame(columns=["Motif", "Motif_Sequence", "Start", "Length", "Strand", "Organism", "Function"])
        
        df = pd.read_csv(
            filename,
            sep="\t",
            header=None,
            names=["Experiment", "Motif", "Motif_Sequence", "Start", "Length", "Strand", "Organism", "Function"]
        )
        return df
    
    elements1 = load_filtered_elements(seq1_name)
    elements2 = load_filtered_elements(seq2_name)
    
    # --- Step 4: Filter elements that overlap with the region ---
    def filter_elements_in_region(df, start_col, end_col):
        if df.empty:
            return df
        
        df['End'] = df['Start'] + df['Length'] - 1
        overlapping = df[
            (df['Start'] <= end_col) & 
            (df['End'] >= start_col)
        ].copy()
        return overlapping
    
    elements1_region = filter_elements_in_region(elements1, start_col, end_col)
    elements2_region = filter_elements_in_region(elements2, start_col, end_col)
    
    # --- Step 5: Assign colors to unique motif names ---
    all_motifs = set()
    if not elements1_region.empty:
        all_motifs.update(elements1_region['Motif'].unique())
    if not elements2_region.empty:
        all_motifs.update(elements2_region['Motif'].unique())
    
    def get_consistent_color(motif_name):
        random.seed(hash(motif_name) % (2**32))
        r = random.randint(180, 240)
        g = random.randint(180, 240)
        b = random.randint(180, 240)
        return f"#{r:02x}{g:02x}{b:02x}"
    
    motif_colors = {motif: get_consistent_color(motif) for motif in sorted(all_motifs)}
    
    # --- Step 6: Calculate name column width ---
    name_col_width = max(len(seq1_name), len(seq2_name)) + 2  # +2 for ": "
    
    # --- Step 7: Create element tracks with PERFECT positioning ---
    def create_element_tracks_html(elements_df, seq_name, is_upper=True):
        if elements_df.empty:
            return ""
        
        element_list = []
        for _, row in elements_df.iterrows():
            elem_start = max(row['Start'], start_col)
            local_start = elem_start - start_col  # 0-based in region
            
            # Add strand in brackets
            strand_display = f" ({row['Strand']})"
            element_list.append({
                'local_start': local_start,
                'motif_seq': row['Motif_Sequence'],
                'motif_name': row['Motif'] + strand_display,
                'color': motif_colors[row['Motif']]
            })
        
        if not element_list:
            return ""
        
        element_list.sort(key=lambda x: x['local_start'])
        
        # Handle overlapping elements
        tracks = []
        for elem in element_list:
            placed = False
            for track in tracks:
                overlaps = False
                for existing in track:
                    # Check if motifs overlap in the display
                    elem_end = elem['local_start'] + len(elem['motif_seq'])
                    existing_end = existing['local_start'] + len(existing['motif_seq'])
                    if not (elem_end <= existing['local_start'] or elem['local_start'] >= existing_end):
                        overlaps = True
                        break
                if not overlaps:
                    track.append(elem)
                    placed = True
                    break
            if not placed:
                tracks.append([elem])
        
        # Generate HTML with precise positioning
        track_html_parts = []
        for track in tracks:
            # Name column spacer
            name_col_span = f'<span style="display: inline-block; width: {name_col_width}ch;"></span>'
            
            # Build the track content
            track_content = []
            
            # Add empty space before first element
            if track[0]['local_start'] > 0:
                empty_width = track[0]['local_start'] + 2
                track_content.append(
                    f'<span style="display: inline-block; width: {empty_width}ch;"></span>'
                )
            
            # Add elements
            for i, elem in enumerate(track):
                # Create the motif sequence span with colored background
                motif_span = (
                    f'<span style="display: inline-block; background-color: {elem["color"]}; '
                    f'padding: 1px 0px; margin: 1px 0; '
                    f'font-family: \'Courier New\', Courier, monospace; '
                    f'font-size: 14px; '
                    f'white-space: nowrap; vertical-align: top;">{elem["motif_seq"]}</span>'
                )
                
                # Create the name span with white background
                name_span = (
                    f'<span style="display: inline-block; background-color: white; '
                    f'padding: 1px 3px; margin: 1px 0; '
                    f'font-family: \'Courier New\', Courier, monospace; '
                    f'font-size: 14px; '
                    f'white-space: pre; vertical-align: top;"> : {elem["motif_name"]}</span>'
                )
                
                track_content.append(motif_span + name_span)
                
                # Add empty space to next element
                if i < len(track) - 1:
                    next_elem = track[i + 1]
                    current_end = elem['local_start'] + len(elem['motif_seq'])
                    gap_width = next_elem['local_start'] - current_end
                    if gap_width > 0:
                        track_content.append(
                            f'<span style="display: inline-block; width: {gap_width}ch;"></span>'
                        )
            
            track_html = name_col_span + ''.join(track_content)
            track_html_parts.append(
                f'<div style="height: 20px; line-height: 20px;">{track_html}</div>'
            )
        
        if is_upper:
            return ''.join(reversed(track_html_parts))
        else:
            return ''.join(track_html_parts)
    
    upper_tracks_html = create_element_tracks_html(elements1_region, seq1_name, is_upper=True)
    lower_tracks_html = create_element_tracks_html(elements2_region, seq2_name, is_upper=False)
    
    # --- Step 8: Create aligned sequences ---
    seq1_html_chars = []
    seq2_html_chars = []
    
    for i in range(region_length):
        char1 = seq1_region[i]
        char2 = seq2_region[i]
        
        if char1 == '-' or char2 == '-':
            if char1 == char2:
                style1 = "background-color: black; color: white;"
                style2 = "background-color: black; color: white;"
            else:
                style1 = "background-color: white; color: black;"
                style2 = "background-color: #d3d3d3; color: black;"
        elif char1.upper() == char2.upper():
            style1 = "background-color: black; color: white;"
            style2 = "background-color: black; color: white;"
        else:
            style1 = "background-color: white; color: black;"
            style2 = "background-color: #d3d3d3; color: black;"
        
        seq1_html_chars.append(
            f'<span style="{style1} display: inline-block; width: 1ch; '
            f'text-align: center; font-family: \'Courier New\', Courier, monospace; '
            f'font-size: 14px;">{char1}</span>'
        )
        seq2_html_chars.append(
            f'<span style="{style2} display: inline-block; width: 1ch; '
            f'text-align: center; font-family: \'Courier New\', Courier, monospace; '
            f'font-size: 14px;">{char2}</span>'
        )
    
    seq1_html = ''.join(seq1_html_chars)
    seq2_html = ''.join(seq2_html_chars)
    
    # --- Step 9: Create position ruler with RIGHT-ALIGNED numbers ---
    ruler_parts = []
    
    for i in range(region_length):
        pos = start_col + i
        if pos % 10 == 0:
            # Show full number, right-aligned to end at this position
            # We'll create a span that contains the number and is right-aligned
            num_str = str(pos)
            ruler_parts.append(
                f'<span style="display: inline-block; width: {len(num_str)}ch; '
                f'text-align: right; font-family: \'Courier New\', Courier, monospace; '
                f'font-size: 14px; color: #666;">{num_str}</span>'
            )
            ruler_parts.append(
                f'<span style="display: inline-block; width: {5 - len(num_str)}ch; '
                f'font-family: \'Courier New\', Courier, monospace; '
                f'font-size: 14px;">&nbsp;</span>'
            )
        else:
            if pos % 5 == 0:
                ruler_parts.append(
                    f'<span style="display: inline-block; width: 1ch; '
                    f'text-align: center; font-family: \'Courier New\', Courier, monospace; '
                    f'font-size: 14px; color: #ccc;">·</span>'
                )
                ruler_parts.append(
                    f'<span style="display: inline-block; width: {4}ch; '
                    f'font-family: \'Courier New\', Courier, monospace; '
                    f'font-size: 14px;">&nbsp;</span>'
                )
    ruler_html = ''.join(ruler_parts)
    
    # --- Step 10: Assemble final HTML ---
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Differential Elements {start_col}-{end_col}</title>
        <style>
            body {{
                font-family: 'Courier New', Courier, monospace;
                font-size: 14px;
                line-height: 1.2;
                margin: 10px;
                background-color: white;
            }}
            .container {{
                max-width: none;
            }}
            .sequence-line {{
                margin: 1px 0;
                white-space: nowrap;
            }}
            .sequence-name {{
                display: inline-block;
                min-width: {name_col_width}ch;
                text-align: right;
                margin-right: 1ch;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            {upper_tracks_html}
            
            <div class="sequence-line">
                <span class="sequence-name">{seq1_name}:</span>
                {seq1_html}
            </div>
            
            <div class="sequence-line">
                <span class="sequence-name">{seq2_name}:</span>
                {seq2_html}
            </div>
            
            {lower_tracks_html}
            
            <div class="sequence-line">
                <span class="sequence-name"></span>
                {ruler_html}
            </div>
        </div>
    </body>
    </html>
    """
    
    # --- Step 11: Display and/or save ---
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"HTML saved to: {output_file}")
    
    display(HTML(html_content))

In [52]:
# Save to HTML file
visualize_differential_elements_html(
    start_col=60, 
    end_col=160, 
    output_file="differential_elements.html"
)

HTML saved to: differential_elements.html
